In [1]:
import json
import sys

from funcx.sdk.client import FuncXClient

In [2]:
fxc = FuncXClient()

In [3]:
func = """
def test_sum_ryan3(event):
    print(event)
    return sum(event['data'])
"""

In [6]:
func_uuid = fxc.register_function("test_sum_ryan3", func, "test_sum_ryan3", 
                                  description="A sum function.")
print(func_uuid)

90abe44b-ef63-4da0-b00d-d157c93e2175


In [7]:
payload = [1,2,3,4,66]

endpoint_uuid = "d342ab25-42e9-4922-b831-37398f67062e"

res = fxc.run(payload, endpoint_uuid, func_uuid)
print(res)

{'endpoint': 'd342ab25-42e9-4922-b831-37398f67062e', 'func': '90abe44b-ef63-4da0-b00d-d157c93e2175', 'is_async': False}
76
